<a href="https://colab.research.google.com/github/jangjung-coding/lettuce_predict_ai/blob/main/lettuce_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (29.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123589 files and direc

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [39]:
%cd /content/drive/MyDrive/googleCloud AI 인재양성 교육/[프로젝트]상추의 생육 환경 생성 AI

/content/drive/MyDrive/googleCloud AI 인재양성 교육/[프로젝트]상추의 생육 환경 생성 AI


In [44]:
all_input_list = sorted(glob.glob('./data/train_input/*.csv'))
all_target_list = sorted(glob.glob('./data/train_target/*.csv'))

test_input_list = sorted(glob.glob('./data/test_input/*.csv'))
test_target_list = sorted(glob.glob('./data/test_target/*.csv'))

In [52]:
sample = pd.read_csv(f'{all_input_list[0]}')
sample.tail(10)

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
662,27,14:00,30.281666,83.190000,495.500000,0.0,252.00,2391.40,18255.19,163368.48,976.332,8737.344,0,0,19231.522,172105.824
663,27,15:00,30.470000,82.856667,485.633333,0.0,252.00,2643.40,18255.19,181623.67,976.332,9713.676,0,0,19231.522,191337.346
664,27,16:00,30.690000,83.160000,477.566667,0.0,245.57,2888.97,18255.19,199878.86,976.332,10690.008,0,0,19231.522,210568.868
665,27,17:00,30.859999,82.635000,468.816667,0.0,245.57,3134.54,18255.19,218134.05,976.332,11666.340,0,0,19231.522,229800.390
666,27,18:00,30.918333,82.628334,465.316667,0.0,252.00,3386.54,18255.19,236389.24,976.332,12642.672,0,0,19231.522,249031.912
667,27,19:00,30.880000,82.255000,464.433333,0.0,252.00,3638.54,18255.19,254644.43,976.332,13619.004,0,0,19231.522,268263.434
668,27,20:00,29.068333,82.506667,534.633333,0.0,126.00,3764.54,618.82,255263.25,33.096,13652.100,0,0,651.916,268915.350
669,27,21:00,28.246667,82.835000,563.433333,0.0,0.00,3764.54,0.00,255263.25,0.000,13652.100,0,0,0.000,268915.350
670,27,22:00,28.005000,82.850000,577.150000,0.0,126.00,3890.54,0.00,255263.25,0.000,13652.100,0,0,0.000,268915.350
671,27,23:00,27.868333,82.453333,588.616667,0.0,0.00,3890.54,0.00,255263.25,0.000,13652.100,0,0,0.000,268915.350


In [53]:
sample = pd.read_csv(f'{all_target_list[0]}')
sample.plot(x='DAT', y='predicted_weight_g')

KeyError: 'predicted_weight_g'